In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def locate_red_leds(image_name):
    def display(img):
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(111)
        ax.imshow(img)

    led_dictionary = {1: [66, 322], 2: [86, 322], 3: [138, 324], 4: [158, 324], 5: [210, 324], 6: [228, 324],7: [278, 324], 8: [300, 324], 9: [348, 324], 10: [368, 324], 11: [418, 326], 12: [438, 326],13: [488, 326], 14: [510, 324], 15: [560, 330], 16: [580, 330]}

    image_path = f" file_location /{image_name}"

    # Read the image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Convert the image from BGR to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define the range of red color in HSV
    lower_red = cv2.inRange(hsv, (0, 100, 100), (15, 255, 255))
    upper_red = cv2.inRange(hsv, (150, 100, 100), (200, 255, 255))
    # Merge the mask of lower and upper red
    mask = cv2.addWeighted(lower_red, 1.0, upper_red, 1.0, 0.0)

    # Apply the mask to the original image
    masked_img = cv2.bitwise_and(img_rgb, img_rgb, mask=mask)
    masked_img = cv2.erode(masked_img, None, iterations=1)
    masked_img = cv2.dilate(masked_img, None, iterations=2)

    # Convert the masked image to grayscale
    gray_img = cv2.cvtColor(masked_img, cv2.COLOR_RGB2GRAY)

    # Perform Hough Circle detection
    circles = cv2.HoughCircles(gray_img, cv2.HOUGH_GRADIENT, 1, 10, param1=155, param2=14, minRadius=0, maxRadius=0)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        print('Number of Red LEDs:', len(circles[0, :]))

        for i, circle in enumerate(circles[0, :], start=1):
            center = (circle[0], circle[1])
            radius = circle[2]
            cv2.circle(img_rgb, center, radius, (0, 0, 0), 2)

            # Find the closest LED in the dictionary
            closest_led = min(led_dictionary, key=lambda x: np.linalg.norm(np.array(led_dictionary[x]) - np.array(center)))
            print(f"LED {closest_led}: {center}")
    else:
        print("No red LEDs detected.")

    # Display the images with circles and LED locations
    display(masked_img)
    display(gray_img)
    #display(img_rgb)
    
    if circles is not None and len(circles[0, :]) == 8:
        return True
    else:
        return False

In [ ]:
image_name = input("Enter the image name (including the extension .jpg): ")
result = locate_red_leds(image_name)

if result:
    print("Result: True")
else:
    print("Check the program or the setup")